# Runtime Test

Before runing this test, please download models from [https://nomeroff.net.ua/models/](https://nomeroff.net.ua/models/) to **./models/** directory. 

In [1]:
# Specify device
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1" 
# os.environ["CUDA_VISIBLE_DEVICES"] = ""  # For CPU inference

In [2]:
 # dynamically grow the memory used on the GPU
from tensorflow.compat.v1.keras.backend import set_session
import tensorflow as tf
#config = tf.compat.v1.ConfigProto()
#config.gpu_options.allow_growth = True 
#sess = tf.compat.v1.Session(config=config)
gpu_options = tf.compat.v1.GPUOptions(per_process_gpu_memory_fraction=0.5)
sess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(gpu_options=gpu_options))
set_session(sess)


In [3]:
# Import all necessary libraries.
import numpy as np
import sys
import glob
import time
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import cv2

# NomeroffNet path
NOMEROFF_NET_DIR = os.path.abspath('../')

# specify the path to Mask_RCNN if you placed it outside Nomeroff-net project
CRAFT_DIR = os.path.join(NOMEROFF_NET_DIR, 'CRAFT-pytorch')

sys.path.append(NOMEROFF_NET_DIR)
sys.path.append(CRAFT_DIR)

In [4]:
from NomeroffNet.MmdetectionDetector import Detector

detector = Detector()
detector.loadModel(cfg_path = '../models/faster_rcnn_r50_caffe_fpn_mstrain_1x_numberpale.py',
                 checkpoint_file = '../models//epoch_12.pth',
                 devie='cuda:0')

Use load_from_local loader


In [5]:
from NomeroffNet.BBoxNpPoints import NpPointsCraft

npPointsCraft = NpPointsCraft()
npPointsCraft.load()

Loading weights from checkpoint (/mnt/ssd/www/nomeroff-netv2/nomeroff-net/weights/craft_mlt_25k.pth)
Loading weights of refiner from checkpoint (/mnt/ssd/www/nomeroff-netv2/nomeroff-net/weights/craft_refiner_CTW1500.pth)


In [6]:
from NomeroffNet.OptionsDetector import OptionsDetector
from NomeroffNet.TextDetector import TextDetector

# Import license plate recognition tools.
from NomeroffNet.RectDetector import  RectDetector

rectDetector = RectDetector()

optionsDetector = OptionsDetector()
optionsDetector.load("latest")

# Initialize text detector.
textDetector = TextDetector({
    "eu_ua_2004_2015": {
        "for_regions": ["eu_ua_2015", "eu_ua_2004"],
        "model_path": "latest"
    },
    "eu_ua_1995": {
        "for_regions": ["eu_ua_1995"],
        "model_path": "latest"
    },
    "eu": {
        "for_regions": ["eu"],
        "model_path": "latest"
    },
    "ru": {
        "for_regions": ["ru", "eu-ua-fake-lnr", "eu-ua-fake-dnr"],
        "model_path": "latest" 
    },
    "kz": {
        "for_regions": ["kz"],
        "model_path": "latest"
    },
    "ge": {
        "for_regions": ["ge"],
        "model_path": "latest"
    },
    "su": {
        "for_regions": ["su"],
        "model_path": "latest"
    }
})

In [7]:
async def test(dirName, fname, min_bbox_acc = 0.5):
    img_path = os.path.join(dirName, fname)
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    #start_time = time.time()
    targetBoxes = detector.detect_bbox([img], min_bbox_acc=min_bbox_acc)[0]
    all_points = npPointsCraft.detect(img, targetBoxes)
    #print("4 point detect", time.time() - start_time)
    
    # for  images/14.jpeg bug
    all_points = [ps for ps in all_points if len(ps)]
    
    zones = rectDetector.get_cv_zonesBGR(img, all_points)
    regionIds, stateIds, countLines = optionsDetector.predict(zones)
    regionNames = optionsDetector.getRegionLabels(regionIds)
    textArr = textDetector.predict(zones, regionNames, countLines)
    return textArr

In [10]:
N = 3

i = 0
j = 0
start_time = time.time()
rootDir = 'images/'
for i in np.arange(N):
    print("pass {}".format(i))
    for dirName, subdirList, fileList in os.walk(rootDir):
        for fname in fileList:
            await test(dirName, fname)
            j += 1
            #print(i, j)
    i += 1
end_time = time.time() - start_time

pass 0
pass 1
pass 2


### Intel(R) Core(TM) i9-9900K CPU @ 3.60GHz + NVIDIA RTX 2080TI

In [11]:
print(f"Processed {j} photos")
print(f"Time {end_time}")
print(f"One photo process {end_time/j} seconds")

Processed 78 photos
Time 27.0348961353302
One photo process 0.34660123250423336 seconds


### Intel(R) Core(TM) i9-9900K CPU @ 3.60GHz + Quadro RTX 8000